In [ ]:
import pandas
import snscrape
from snscrape.modules.twitter import TwitterSearchScraper


In [ ]:
import sqlite3

con = sqlite3.connect('./dataset/dataset_db.db')
cur = con.cursor()
cur.execute('''
                CREATE TABLE IF NOT EXISTS tweet (
                    id              INTEGER PRIMARY KEY AUTOINCREMENT,
                    url             TEXT NOT NULL,
                    date            TEXT NOT NULL,
                    like_count      INTEGER NOT NULL,
                    reply_count     INTEGER NOT NULL,
                    retweet_count   INTEGER NOT NULL
            );''')
cur.execute('''
                CREATE TABLE IF NOT EXISTS tweet_media (
                    id              INTEGER PRIMARY KEY AUTOINCREMENT,
                    tweet_id        INTEGER NOT NULL,
                    type            TEXT NOT NULL,
                    url             TEXT NOT NULL
            );''')
cur.execute('''
                CREATE TABLE IF NOT EXISTS media (
                    id              INTEGER PRIMARY KEY AUTOINCREMENT,
                    url             TEXT NOT NULL,
                    model           TEXT NOT NULL,    
                    status          TEXT,
                    media_type      TEXT NOT NULL,
                    source          TEXT NOT NULL
            );''')

def insert_into_table(db: str, table_name: str, data: list, return_rowid: bool):
    con = sqlite3.connect(db)
    cur = con.cursor()
    
    sql = f"INSERT OR IGNORE INTO {table_name} VALUES ({','.join(['?' for _ in data[0]])})"
    cur.executemany(sql, data)
    
    if return_rowid:
        cur.execute(f"SELECT MAX(id) FROM {table_name}")
        result = cur.fetchone()
        last_rowid = result[0]
        con.commit()
        con.close()
        return last_rowid

    con.commit()
    con.close()

def update_table(db: str, table_name: str, set_values_dict: dict, where_clause: str):
    con = sqlite3.connect(db)
    cur = con.cursor()

    set_clause = ', '.join([f"{col} = ?" for col in set_values_dict.keys()])
    sql = f"UPDATE {table_name} SET {set_clause} WHERE {where_clause}"
    cur.execute(sql, tuple(set_values_dict.values()))

    con.commit()
    con.close()

In [ ]:
import snscrape.modules.twitter as sntwitter
tweet_list = []
for i,tweet in enumerate(sntwitter.TwitterUserScraper('IgnacioBusso').get_items()):
    if i>50:
        break
    # print(vars(tweet))
    if tweet.media:
        print(tweet.media)

    # tweet_list.append(tweet)

In [ ]:
import snscrape.modules.twitter as sntwitter

def get_specific_tweet(tweet_id):
    print(tweet_id)
    tweets_list = []
    # Using TwitterSearchScraper to scrape data and append tweets to list
 
    for i,tweet in enumerate(sntwitter.TwitterTweetScraper(tweetId=1656853595352883202,mode=sntwitter.TwitterTweetScraperMode.SINGLE).get_items()):
        # print(tweet)
        # print(tweet.rawContent)
        # print(vars(tweet))
        print(tweet.media)
        print(tweet.media[0])
        print(tweet.media[0].fullUrl)
        # print(type(tweet.media))
        # print(type(tweet.media[0]))
get_specific_tweet(1656853595352883202)

# GALLERY, 4 IMAGES: 
#       https://twitter.com/TheGreyPatriot_/status/1657313944317427713
#       https://twitter.com/PucaraD/status/1657009349489770496
# 1 IMAGE            
#       1656853595352883202
# VIDEO              

In [ ]:
query_vehicle_list = [  # (model, query content)
    # ('M113', 'M113'),
    # ('MT-LB','MT-LB OR MT-LBV OR MT-LBVM OR MT-LBVMK OR MT-LBVM/K OR MT-LBu'),
    # ('BTR-80','BTR-80 OR BTR-82'),
    # ('BTR-82A','BTR-80A OR BTR-82A'),
    # ('BMP-1','BMP-1'),
    # ('BMP-2','BMP-2 OR BMP-2M'),
    # ('BMP-3','BMP-3 OR BMP-3M'),
    # ('T-62','T-62 OR T-62M OR T-62MV'),
    # ('T-64','T-64 OR T-64A OR T-64B OR T-64BV OR T-64B1M OR T-64BM OR T-64BM2'),
    ('T-72','T-72 OR T-72A OR T-72AV OR T-72AMT OR T-72B OR T-72BA OR T-72B3 OR T-72B3 OR T-72M OR T-72M1'),
    # ('T-80','T-80 OR T-80B OR T-80BV OR T-80U OR T-80BVM'),
    # ('T-90','T-90 OR T-90A OR T-90M'),
    # ('2S1','2S1 OR Gvozdika'),
    # ('2S3','2S3 OR Akatsiya'),
    # ('2S19','2S19 OR 2S19M OR 2S19M1 OR 2S19M2 OR Msta OR Msta-S OR Msta-SM2'),
    # ('BM-21','BM-21'),
]

query_terms_list = [
    'russian', 
    'ukrainian', 
    # 'russia', 
    # 'ukraine', 
    # 'DNR', 
    # 'DPR', 
    # 'LNR', 
    # 'LPR', 
    # 'captured', 
    # 'abandoned', 
    # 'damaged', 
    # 'destroyed', 
    # 'kherson', 
    # 'kharkiv', 
    # 'donetsk', 
    # 'severodonetsk', 
    # 'luhansk', 
    # 'lugansk', 
    # 'donbass',
    # 'Dnieper', 
    # 'Dnipro', 
    # 'izium', 
    # 'izyum ',
    # 'bakhmut', 
    # 'offensive', 
    # 'attack', 
    # 'repulsed'
]

In [ ]:
import snscrape.modules.twitter as sntwitter
query = '(T-90 ukraine ukranian) filter:links lang:en'
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    print(tweet.date)

In [ ]:
import snscrape.modules.twitter as sntwitter

date_since = '2022-02-23'
date_until = '2023-02-25'
vehicle_type = 'T-72'
filters = 'filter:videos lang:en'

# url_counter = 0
# url_list = []
# url_list = read_list(vehicle_type)

for term in query_terms_list:
    query = f'{vehicle_type} {term} since:{date_since} until:{date_until} {filters}'
    print(query)
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if tweet.media:
            tweet_data = [None, tweet.url, str(tweet.date).split('+')[0], tweet.likeCount, tweet.replyCount, tweet.retweetCount]
            last_rowid = insert_into_table('./dataset/dataset_db.db', 'tweet', [tweet_data], True)
            for medium in tweet.media:
                if isinstance(medium, sntwitter.Photo):
                    pass
                    # Add to tweet_media table
                    image_url = medium.fullUrl.replace('?format=', '.', 1).rsplit('&', 1)[0]
                    media_data = [None, last_rowid, 'image', image_url]
                    insert_into_table('./dataset/dataset_db.db', 'tweet_media', [media_data], False)

                    # Add to JSON files
                    # url_list.append([image_url,'unknown','image','twitter'])
                elif isinstance(medium, sntwitter.Video):

                    # Filter out variants 'video' in its content type
                    video_list = [variant for variant in medium.variants if 'video' in str(variant.contentType)]

                    # Sort the videos by bitrate (descending order)
                    sorted_video_list = sorted(video_list, key=lambda v: v.bitrate, reverse=True)

                    # Get the URL of the variant with the highest bitrate
                    if sorted_video_list:
                        video_url = sorted_video_list[0].url
                    else:
                        video_url = None

                    # Add to tweet_media table
                    media_data = [None, last_rowid, 'video', video_url]
                    insert_into_table('./dataset/dataset_db.db', 'tweet_media', [media_data], False)

                    # Add to JSON files
                    # url_list.append([video_url,'unknown','image','twitter'])
                elif isinstance(medium, sntwitter.Gif):
                    # print(medium.variants[0].url)
                    pass
                else:
                    # print(medium.variants[0].url)
                    pass

                        
                    # elif 'twitter.com/'  in item.url:


        # print(tweet.url)
        # try:
        #     media = tweet.media[0].fullUrl # .previewUrl if you want previewUrl
        #     print(media)
        # except:
        #     media = tweet.media # or None or '' or any default value 
        #     print(media)

 
query = 'T-72 filter:images since:2022-02-23 until:2022-12-31' 
# since:2023-01-01 until:2023-02-01' lang:es 
# filter:image filter:images filter:videos filter:links filter:media 

print(query)

# query = 'gm filter:videos lang:en'

In [ ]:
for row in cur.execute("SELECT * FROM tweet"):
    print(row)

In [ ]:
for row in cur.execute("SELECT * FROM tweet_media"):
    print(row)

In [ ]:
# for medium in tweet.media:
# 	if medium.type == 'photo':
# 		# Do something with medium.fullUrl
# 	elif medium.type == 'video':
# 		# Do something with one of the URLs in medium.variants

# if tweet.media:
#     for medium in tweet.media:
#         if isinstance(medium, snscrape.modules.twitter.Photo):
#             print(medium.fullUrl.rsplit('=', 1)[0] + '=orig')  # Replace =large with =orig
#         elif isinstance(medium, (snscrape.modules.twitter.Video, snscrape.modules.twitter.Gif)):
#             # do something with medium.variants, I guess

In [ ]:
# VIDEO MEDIUM FORMAT
# Video(
# 	thumbnailUrl='https://pbs.twimg.com/ext_tw_video_thumb/1496897376707129345/pu/img/1xIC4FYQ6nhQT5iF.jpg', 
# 	variants=[
# 		VideoVariant(url='https://video.twimg.com/ext_tw_video/1496897376707129345/pu/vid/480x854/LO7Dab3-krcbozuE.mp4?tag=12', contentType='video/mp4', bitrate=950000), 
# 		VideoVariant(url='https://video.twimg.com/ext_tw_video/1496897376707129345/pu/pl/G_rCkEGupzECbXvF.m3u8?tag=12&container=fmp4', contentType='application/x-mpegURL', bitrate=None), 
# 		VideoVariant(url='https://video.twimg.com/ext_tw_video/1496897376707129345/pu/vid/320x568/xXfRvaSVqoPE_WNy.mp4?tag=12', contentType='video/mp4', bitrate=632000)], 
# 	duration=23.798, 
# 	views=50, altText=None)



# Video(
# 	thumbnailUrl='https://pbs.twimg.com/ext_tw_video_thumb/1496897160935358466/pu/img/-zqKLLzc10jZWxHL.jpg', 
# 	variants=[
# 		VideoVariant(url='https://video.twimg.com/ext_tw_video/1496897160935358466/pu/vid/592x1280/83ZS6AHbqiB-4VY0.mp4?tag=12', contentType='video/mp4', bitrate=2176000), 
# 		VideoVariant(url='https://video.twimg.com/ext_tw_video/1496897160935358466/pu/vid/480x1036/c4ptH8swrfVHKOPv.mp4?tag=12', contentType='video/mp4', bitrate=950000), 
# 		VideoVariant(url='https://video.twimg.com/ext_tw_video/1496897160935358466/pu/vid/320x690/en8hFf9jSDIW-kET.mp4?tag=12', contentType='video/mp4', bitrate=632000), 
# 		VideoVariant(url='https://video.twimg.com/ext_tw_video/1496897160935358466/pu/pl/7p64JbsupNsijgRT.m3u8?tag=12&container=fmp4', contentType='application/x-mpegURL', bitrate=None)], 
# 	duration=11.261, 
# 	views=450, 
# 	altText=None)

In [ ]:
for tweet in tweet_list:
    print(tweet)

In [ ]:
scraper = TwitterSearchScraper(query)

result = []

for i, item in enumerate(scraper.get_items()):
        result.append(item)
        # print(tweet.media)
        # print(tweet.media[0].fullUrl)
        try:
            media = tweet.media[0].fullUrl # .previewUrl if you want previewUrl
            print(media)
        except:
            media = tweet.media # or None or '' or any default value 
            print(media)
        if i == 100:
                break
        
print(len(result))
for i in result:
    print(i)